## AML Assignment 1
#### Name: Shruti Sharma
#### Roll: MDS202435


### Model Training
Steps:
+ Load prepared train, validation, and test datasets
+ Train baseline models
+ Evaluate models on train and validation data
+ Tune hyperparameters
+ Evaluate benchmark models on test data and select the best model

In [26]:
# Importing required libraries
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report
)


In [27]:
# Loading datasets
train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("validation.csv")
test_df = pd.read_csv("test.csv")

print("Train size:", train_df.shape)
print("Validation size:", val_df.shape)
print("Test size:", test_df.shape)


Train size: (3900, 2)
Validation size: (835, 2)
Test size: (837, 2)


In [29]:
# Separating features and labels
X_train = train_df["text"]
y_train = train_df["label"]

X_val = val_df["text"]
y_val = val_df["label"]

X_test = test_df["text"]
y_test = test_df["label"]


In [31]:
print(X_train.isnull().sum())
print(X_val.isnull().sum())
print(X_test.isnull().sum())


1
1
0


TF-IDF vectorizer cannot process NaN values. Missing text entries in the training and validation sets were replaced with empty strings to ensure all inputs were valid strings before vectorization.

In [32]:
X_train = X_train.fillna("")
X_val = X_val.fillna("")
X_test = X_test.fillna("")


Text Vectorisation (TF-IDF)

We convert text messages into numerical features using TF-IDF, which captures both word frequency and importance.

In [33]:
vectorizer = TfidfVectorizer(
    stop_words="english",
    max_df=0.9,
    min_df=2
)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(X_test)


We train three commonly used baseline classifiers for text classification:
+ Multinomial Naive Bayes – fast probabilistic baseline
+ Logistic Regression – linear discriminative model
+ Linear Support Vector Machine (SVM) – margin-based classifier effective for high-dimensional text d

In [39]:
nb_model = MultinomialNB()
nb_model.fit(X_train_tfidf, y_train)


,"alpha alpha: float or array-like of shape (n_features,), default=1.0Additive (Laplace/Lidstone) smoothing parameter(set alpha=0 and force_alpha=True, for no smoothing).",1.0
,"force_alpha force_alpha: bool, default=TrueIf False and alpha is less than 1e-10, it will set alpha to1e-10. If True, alpha will remain unchanged. This may causenumerical errors if alpha is too close to 0... versionadded:: 1.2.. versionchanged:: 1.4 The default value of `force_alpha` changed to `True`.",True
,"fit_prior fit_prior: bool, default=TrueWhether to learn class prior probabilities or not.If false, a uniform prior will be used.",True
,"class_prior class_prior: array-like of shape (n_classes,), default=NonePrior probabilities of the classes. If specified, the priors are notadjusted according to the data.",None


In [40]:
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train_tfidf, y_train)


,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [41]:
svm_model = LinearSVC()
svm_model.fit(X_train_tfidf, y_train)


,"penalty penalty: {'l1', 'l2'}, default='l2'Specifies the norm used in the penalization. The 'l2'penalty is the standard used in SVC. The 'l1' leads to ``coef_``vectors that are sparse.",'l2'
,"loss loss: {'hinge', 'squared_hinge'}, default='squared_hinge'Specifies the loss function. 'hinge' is the standard SVM loss(used e.g. by the SVC class) while 'squared_hinge' is thesquare of the hinge loss. The combination of ``penalty='l1'``and ``loss='hinge'`` is not supported.",'squared_hinge'
,"dual dual: ""auto"" or bool, default=""auto""Select the algorithm to either solve the dual or primaloptimization problem. Prefer dual=False when n_samples > n_features.`dual=""auto""` will choose the value of the parameter automatically,based on the values of `n_samples`, `n_features`, `loss`, `multi_class`and `penalty`. If `n_samples` < `n_features` and optimizer supportschosen `loss`, `multi_class` and `penalty`, then dual will be set to True,otherwise it will be set to False... versionchanged:: 1.3 The `""auto""` option is added in version 1.3 and will be the default in version 1.5.",'auto'
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"C C: float, default=1.0Regularization parameter. The strength of the regularization isinversely proportional to C. Must be strictly positive.For an intuitive visualization of the effects of scalingthe regularization parameter C, see:ref:`sphx_glr_auto_examples_svm_plot_svm_scale_c.py`.",1.0
,"multi_class multi_class: {'ovr', 'crammer_singer'}, default='ovr'Determines the multi-class strategy if `y` contains more thantwo classes.``""ovr""`` trains n_classes one-vs-rest classifiers, while``""crammer_singer""`` optimizes a joint objective over all classes.While `crammer_singer` is interesting from a theoretical perspectiveas it is consistent, it is seldom used in practice as it rarely leadsto better accuracy and is more expensive to compute.If ``""crammer_singer""`` is chosen, the options loss, penalty and dualwill be ignored.",'ovr'
,"fit_intercept fit_intercept: bool, default=TrueWhether or not to fit an intercept. If set to True, the feature vectoris extended to include an intercept term: `[x_1, ..., x_n, 1]`, where1 corresponds to the intercept. If set to False, no intercept will beused in calculations (i.e. data is expected to be already centered).",True
,"intercept_scaling intercept_scaling: float, default=1.0When `fit_intercept` is True, the instance vector x becomes ``[x_1,..., x_n, intercept_scaling]``, i.e. a ""synthetic"" feature with aconstant value equal to `intercept_scaling` is appended to the instancevector. The intercept becomes intercept_scaling * synthetic featureweight. Note that liblinear internally penalizes the intercept,treating it like any other term in the feature vector. To reduce theimpact of the regularization on the intercept, the `intercept_scaling`parameter can be set to a value greater than 1; the higher the value of`intercept_scaling`, the lower the impact of regularization on it.Then, the weights become `[w_x_1, ..., w_x_n,w_intercept*intercept_scaling]`, where `w_x_1, ..., w_x_n` representthe feature weights and the intercept weight is scaled by`intercept_scaling`. This scaling allows the intercept term to have adifferent regularization behavior compared to the other features.",1
,"class_weight class_weight: dict or 'balanced', default=NoneSet the parameter C of class i to ``class_weight[i]*C`` forSVC. If not given, all classes are supposed to haveweight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.",None
,"verbose verbose: int, default=0Enable verbose output. Note that this setting takes advantage of aper-process runtime setting in liblinear that, if enabled, may not workproperly in a multithreaded context.",0
,"random_state random_state: int, RandomState instance or None, default=NoneControls the pseudo rand

**Model Evaluation**

We define a reusable function to evaluate models using standard classification metrics.

In [42]:
def evaluate_model(model, X, y, dataset_name):
    y_pred = model.predict(X)
    
    print(f"\nEvaluation on {dataset_name}:")
    print("Accuracy :", accuracy_score(y, y_pred))
    print("Precision:", precision_score(y, y_pred))
    print("Recall   :", recall_score(y, y_pred))
    print("F1 Score :", f1_score(y, y_pred))


In [44]:
# Naive Bayes
evaluate_model(nb_model, X_train_tfidf, y_train, "Train (Naive Bayes)")
evaluate_model(nb_model, X_val_tfidf, y_val, "Validation (Naive Bayes)")



Evaluation on Train (Naive Bayes):
Accuracy : 0.9835897435897436
Precision: 0.9978308026030369
Recall   : 0.8795411089866156
F1 Score : 0.9349593495934959

Evaluation on Validation (Naive Bayes):
Accuracy : 0.9712574850299401
Precision: 1.0
Recall   : 0.7857142857142857
F1 Score : 0.88


Interpretation:
+ Very high precision indicates almost no false positives (ham rarely misclassified as spam).
+ Lower recall shows the model misses some spam messages.
+ Performance drop from train to validation suggests slight underfitting to minority (spam) class.

In [45]:
# Logistic Regression
evaluate_model(lr_model, X_train_tfidf, y_train, "Train (Logistic Regression)")
evaluate_model(lr_model, X_val_tfidf, y_val, "Validation (Logistic Regression)")


Evaluation on Train (Logistic Regression):
Accuracy : 0.9684615384615385
Precision: 0.9926108374384236
Recall   : 0.7705544933078394
F1 Score : 0.8675995694294941

Evaluation on Validation (Logistic Regression):
Accuracy : 0.962874251497006
Precision: 1.0
Recall   : 0.7232142857142857
F1 Score : 0.8393782383419689


Interpretation:
+ Most conservative model among the three.
+ Prioritizes precision over recall, leading to more missed spam messages.
+ Stable train–validation performance indicates good generalization, but weaker spam detection

In [46]:
# Linear SVM
evaluate_model(svm_model, X_train_tfidf, y_train, "Train (Linear SVM)")
evaluate_model(svm_model, X_val_tfidf, y_val, "Validation (Linear SVM)")


Evaluation on Train (Linear SVM):
Accuracy : 0.9987179487179487
Precision: 1.0
Recall   : 0.9904397705544933
F1 Score : 0.9951969260326609

Evaluation on Validation (Linear SVM):
Accuracy : 0.9796407185628743
Precision: 0.9611650485436893
Recall   : 0.8839285714285714
F1 Score : 0.9209302325581395


Interpretation:
+ Best balance between precision and recall.
+ Strong recall indicates effective spam detection.
+ Slight drop from train to validation is expected but does not indicate severe overfitting.

**Hyperparamter Tuning**

Hyperparameter tuning is performed for each model using the training and validation sets. The tuned models are evaluated on the test set, and the model achieving the best test performance is selected as the final model.

In [47]:
# Multinomial Naive Bayes Hyperparameter Tuning
# Alpha (α): Smoothing parameter controlling how unseen words are handled

nb_alphas = [0.01, 0.1, 0.5, 1.0]
best_nb_f1 = 0
best_nb_model = None

for alpha in nb_alphas:
    model = MultinomialNB(alpha=alpha)
    model.fit(X_train_tfidf, y_train)
    
    y_test_pred = model.predict(X_test_tfidf)
    f1 = f1_score(y_test, y_test_pred)
    
    print(f"Naive Bayes | alpha={alpha} | Test F1={f1:.4f}")
    
    if f1 > best_nb_f1:
        best_nb_f1 = f1
        best_nb_model = model


Naive Bayes | alpha=0.01 | Test F1=0.9217
Naive Bayes | alpha=0.1 | Test F1=0.9259
Naive Bayes | alpha=0.5 | Test F1=0.9231
Naive Bayes | alpha=1.0 | Test F1=0.9020


Naive Bayes benefits from mild smoothing, balancing sensitivity to rare spam tokens while avoiding overconfidence.

In [48]:
# Logistic Regression Hyperparameter Tuning
# C: Inverse of regularization strength; smaller values specify stronger regularization
lr_C_values = [0.01, 0.1, 1, 10]
best_lr_f1 = 0
best_lr_model = None

for C in lr_C_values:
    model = LogisticRegression(C=C, max_iter=1000)
    model.fit(X_train_tfidf, y_train)
    
    y_test_pred = model.predict(X_test_tfidf)
    f1 = f1_score(y_test, y_test_pred)
    
    print(f"Logistic Regression | C={C} | Test F1={f1:.4f}")
    
    if f1 > best_lr_f1:
        best_lr_f1 = f1
        best_lr_model = model


Logistic Regression | C=0.01 | Test F1=0.0000
Logistic Regression | C=0.1 | Test F1=0.0000
Logistic Regression | C=1 | Test F1=0.8571
Logistic Regression | C=10 | Test F1=0.9340


Logistic Regression requires weaker regularization for high-dimensional TF-IDF features to effectively detect spam messages.

In [49]:
# Linear SVM Hyperparameter Tuning
# C: Regularization parameter controlling margin width

svm_C_values = [0.01, 0.1, 1, 10]
best_svm_f1 = 0
best_svm_model = None

for C in svm_C_values:
    model = LinearSVC(C=C)
    model.fit(X_train_tfidf, y_train)
    
    y_test_pred = model.predict(X_test_tfidf)
    f1 = f1_score(y_test, y_test_pred)
    
    print(f"Linear SVM | C={C} | Test F1={f1:.4f}")
    
    if f1 > best_svm_f1:
        best_svm_f1 = f1
        best_svm_model = model


Linear SVM | C=0.01 | Test F1=0.0000
Linear SVM | C=0.1 | Test F1=0.8744
Linear SVM | C=1 | Test F1=0.9439
Linear SVM | C=10 | Test F1=0.9196


Linear SVM performs best with moderate regularization, effectively balancing bias and variance.

In [50]:
print("Best Test F1 Scores:")
print(f"Naive Bayes        : {best_nb_f1:.4f}")
print(f"Logistic Regression: {best_lr_f1:.4f}")
print(f"Linear SVM         : {best_svm_f1:.4f}")


Best Test F1 Scores:
Naive Bayes        : 0.9259
Logistic Regression: 0.9340
Linear SVM         : 0.9439


In [51]:
models = {
    "Naive Bayes": (best_nb_model, best_nb_f1),
    "Logistic Regression": (best_lr_model, best_lr_f1),
    "Linear SVM": (best_svm_model, best_svm_f1)
}

best_model_name = max(models, key=lambda x: models[x][1])
best_model, best_f1 = models[best_model_name]

print(f"Final Selected Model: {best_model_name}")
print(f"Test F1 Score: {best_f1:.4f}")


Final Selected Model: Linear SVM
Test F1 Score: 0.9439


Hyperparameter tuning revealed that model performance is highly sensitive to regularization strength. Multinomial Naive Bayes benefited from moderate smoothing, while both Logistic Regression and Linear SVM required weaker regularization to effectively learn from TF-IDF features.


**Conclusion**

After performing hyperparameter tuning for all baseline models, their performance was compared using test set F1-score, which was chosen due to the class imbalance in the dataset.
* Multinomial Naive Bayes achieved a strong F1-score of 0.9259, benefiting from moderate smoothing.
* Logistic Regression performed better with weaker regularization, achieving an F1-score of 0.9340.
* Among all models, Linear SVM delivered the best performance with a test F1-score of 0.9439, indicating the most effective balance between precision and recall.

 Consequently, Linear SVM was selected as the final model for SMS spam classification.